## This demo app shows:
* How to use LangChain's YoutubeLoader to retrieve the caption in a YouTube video
* How to ask Llama 3 to summarize the content (per the Llama's input size limit) of the video in a naive way using LangChain's stuff method
* How to bypass the limit of Llama 3's max input token size by using a more sophisticated way using LangChain's map_reduce and refine methods - see [here](https://python.langchain.com/docs/use_cases/summarization) for more info

We start by installing the necessary packages:
- [youtube-transcript-api](https://pypi.org/project/youtube-transcript-api/) API to get transcript/subtitles of a YouTube video
- [langchain](https://python.langchain.com/docs/get_started/introduction) provides necessary RAG tools for this demo
- [tiktoken](https://github.com/openai/tiktoken) BytePair Encoding tokenizer
- [pytube](https://pytube.io/en/latest/) Utility for downloading YouTube videos

**Note** This example uses OctoAI to host the Llama 3 model. If you have not set up/or used OctoAI before, we suggest you take a look at the [HelloLlamaCloud](HelloLlamaCloud.ipynb) example for information on how to set up OctoAI before continuing with this example.
If you do not want to use OctoAI, you will need to make some changes to this notebook as you go along.

In [ ]:
!pip install langchain==0.1.19 youtube-transcript-api tiktoken pytube

Let's first load a long (2:47:16) YouTube video (Lex Fridman with Yann Lecun: Meta AI, Open Source, Limits of LLMs, AGI & the Future of AI) transcript using the YoutubeLoader.

In [ ]:
from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=5t1vTLU7s40", add_video_info=True
)

In [ ]:
# load the youtube video caption into Documents
docs = loader.load()

In [ ]:
# check the docs length and content
len(docs[0].page_content), docs[0].page_content[:300]

You should see 142689 returned for the doc character length, which is about 30k words or 40k tokens, beyond the 8k context length limit of Llama 3. You'll see how to summarize a text longer than the limit.

**Note**: We are using OctoAI in this example to host our Llama 3 model so you will need to get a OctoAI token.

To get the OctoAI token:

- You will need to first sign in with OctoAI with your github account
- Then create a free API token [here](https://octo.ai/docs/getting-started/how-to-create-an-octoai-access-token) that you can use for a while (a month or $10 in OctoAI credits, whichever one runs out first)

After the free trial ends, you will need to enter billing info to continue to use Llama2 hosted on OctoAI.

In [ ]:
# enter your OctoAI API token, or you can use local Llama. See README for more info
from getpass import getpass
import os

OCTOAI_API_TOKEN = getpass()
os.environ["OCTOAI_API_TOKEN"] = OCTOAI_API_TOKEN

Next we call the Llama 3 model from OctoAI. In this example we will use the Llama 3 8b instruct model. You can find more on Llama models on the [OctoAI text generation solution page](https://octoai.cloud/text).

At the time of writing this notebook the following Llama models are available on OctoAI:
* meta-llama-3-8b-instruct
* meta-llama-3-70b-instruct
* codellama-7b-instruct
* codellama-13b-instruct
* codellama-34b-instruct
* llama-2-13b-chat
* llama-2-70b-chat
* llamaguard-7b

In [ ]:
from langchain.llms.octoai_endpoint import OctoAIEndpoint

llama3_8b = "meta-llama-3-8b-instruct"
llm = OctoAIEndpoint(
    model=llama3_8b,
    max_tokens=500,
    temperature=0.01
)

Once everything is set up, we prompt Llama 3 to summarize the first 4000 characters of the transcript for us.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template = "Give me a summary of the text below: {text}?"
prompt = PromptTemplate(
    input_variables=["text"], template=prompt_template
)
chain = prompt | llm

# be careful of the input text length sent to LLM
text = docs[0].page_content[:10000]
summary = chain.invoke(text)

# Note: The context length of 8k tokens in Llama 3 is roughly 6000-7000 words or 32k characters
print(summary)

If you try the whole content which has over 142k characters, about 40k tokens, which exceeds the 8k limit, you'll get an empty result (OctoAI used to return an error "BadRequestError: The token count (32704) of your prompt (32204) + your setting of `max_tokens` (500) cannot exceed this model's context length (8192).").

In [ ]:
# this will generate an empty result because the input exceeds Llama 3's context length limit
text = docs[0].page_content
summary = llm.invoke(f"Give me a summary of the text below: {text}.")
print(summary)

To fix this, you can use LangChain's load_summarize_chain method (detail [here](https://python.langchain.com/docs/use_cases/summarization)).

First you'll create splits or sub-documents of the original content, then use the LangChain's `load_summarize_chain` with the `refine` or `map_reduce type`.

Because this may involve many calls to Llama 3, it'd be great to set up a quick free LangChain API key [here](https://smith.langchain.com/settings), run the following cell to set up necessary environment variables, and check the logs on [LangSmith](https://docs.smith.langchain.com/) during and after the run.

In [ ]:
import os
os.environ["LANGCHAIN_API_KEY"] = "your_langchain_api_key"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Video Summary with Llama 3"

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# we need to split the long input text
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [ ]:
# check the splitted docs lengths
len(split_docs), len(docs), len(split_docs[0].page_content), len(docs[0].page_content)

The `refine` type implements the following steps under the hood:

1. Call Llama 3 on the first sub-document to generate a concise summary;
2. Loop over each subsequent sub-document, pass the previous summary with the current sub-document to generate a refined new summary;
3. Return the final summary generated on the final sub-document as the final answer - the summary of the whole content.

An example prompt template for each call in step 2, which gets used under the hood by LangChain, is:

```
Your job is to produce a final summary.
We have provided an existing summary up to a certain point:
<previous_summary>
Refine the existing summary (only if needed) with some more content below:
<new_content>
```

**Note**: The following call will make 33 calls to Llama 3 and genereate the final summary in about 10 minutes.

In [ ]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="refine")
print(chain.run(split_docs))

You can also set `chain_type` to `map_reduce` to generate the summary of the entire content using the standard map and reduce method, which works behind the scene by first mapping each split document to a sub-summary via a call to LLM, then combines all those sub-summaries into a single final summary by yet another call to LLM.

**Note**: The following call takes about 3 minutes and all the calls to Llama 3.

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
print(chain.run(split_docs))